In [2]:
import numpy as np
from itertools import combinations, product
from myf import calculate_f_jk, calculate_g_x
from joblib import Parallel, delayed
import os
import pickle
from tslearn.barycenters import softdtw_barycenter
import torch

In [ ]:
# torch.cuda.set_device(1)  # Manually set the active device
# device = torch.device("cuda:1")
# tensor = torch.randn(3, 3).to(device)  # Move tensor to GPU 1
# print(torch.cuda.current_device())  # Should return 1 if using GPU 2

In [2]:
# Set default device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda:1


In [ ]:
def single_motif(motifs, profiles, tensor, mask_tensor, gamma, tau, n_jobs=-1):

    # Convert masked array data to a PyTorch tensor
    # tensor = torch.tensor(masked_arr.filled(float("inf")), dtype=torch.float32).to("cuda")# Use 'inf' for masked values

    # Convert the mask to a boolean tensor
    # mask_tensor = torch.tensor(masked_arr.mask, dtype=torch.bool).to("cuda")#mask only, 0 non mask, 1 mask

    # make sure for min(dist(profiles[row], motif_centers), dist(profiles[col], motif_centers))
    motif_centers_idx = []  # has all the motif centers
    motif_dist_threshold = (
        []
    )  # the corresponding dist_threshold radius min_score+tau + tau(min radius thresold of new motif)

    # Get minimum score
    while True:
        if not motifs:  # Empty dictionary
            # Find the minimum value while ignoring masked elements
            min_value = torch.min(
                tensor[~mask_tensor]
            )  # Mask out ignored elements #cuda

            # Get the index of the minimum value (1D index)
            argmin_index = torch.argmin(tensor[~mask_tensor])

            # Convert back to 2D indices
            valid_indices = torch.nonzero(~mask_tensor)  # Get all valid positions
            chosen_row, chosen_col = valid_indices[
                argmin_index
            ]  # Retrieve corresponding (row, col)

            if min_value >= gamma:
                return None, tensor, mask_tensor

        else:

            if not motif_centers_idx:  # initiation

                # should change centers_idx and dist_thresold to tensor after

                for motif in motifs:
                    motif_centers_idx.append(motif["pair_idx"][0])
                    motif_dist_threshold.append(motif["score"] + 2 * tau)
                    motif_centers_idx.append(motif["pair_idx"][1])
                    motif_dist_threshold.append(motif["score"] + 2 * tau)

                motif_centers_idx_tensor = torch.tensor(
                    motif_centers_idx, dtype=torch.long
                ).to("cuda")

                motif_dist_thresold_tensor = torch.tensor(
                    motif_dist_threshold, dtype=torch.float32
                ).to("cuda")

            # Get indices where tensor values < gamma and NOT masked
            valid_indices = torch.nonzero(
                (tensor < gamma) & (~mask_tensor), as_tuple=False
            )

            # Extract row and column separately
            valid_rows = valid_indices[:, 0]  # First column -> row indices
            valid_cols = valid_indices[:, 1]  # Second column -> column indices

            # Compute distances for rows to motif centers
            row_distance_first = tensor[
                valid_rows.unsqueeze(1), motif_centers_idx_tensor
            ]  # Shape (n, k)n: rows cand, k: motifs
            # valid_rows: real row indices

            row_distance_second = tensor[
                motif_centers_idx_tensor.unsqueeze(1), valid_rows
            ].T  # Shape (k, n) -> (n, k)

            # Stack along final dim
            row_stacked = torch.stack(
                [row_distance_first, row_distance_second], dim=-1
            )  # Shape (n, k, 2)

            # Compute min across the last dimension
            final_row_min = torch.min(row_stacked, dim=-1).values  # Shape (n, k)
            # min dist of corresponding row to each motif center(representation ts)
            # corresponding idx in row_tensor is real row idx

            col_distance_first = tensor[
                valid_cols.unsqueeze(1), motif_centers_idx_tensor
            ]
            col_distance_second = tensor[
                motif_centers_idx_tensor.unsqueeze(1), valid_cols
            ].T
            col_stacked = torch.stack([col_distance_first, col_distance_second], dim=-1)
            final_col_min = torch.min(col_stacked, dim=-1).values

            # Check where all row values are greater than or equal to their threshold
            rows_threshold = torch.all(
                final_row_min >= motif_dist_thresold_tensor, dim=1
            )  # Shape (n,), bool
            cols_threshold = torch.all(
                final_col_min >= motif_dist_thresold_tensor, dim=1
            )

            # Stack them together into shape (n, 2)
            stacked_valid = torch.stack(
                [rows_threshold, cols_threshold], dim=1
            )  # Shape (n, 2)

            # Filter where both conditions are True
            filtered_in_indices = torch.nonzero(
                torch.all(stacked_valid, dim=1), as_tuple=True
            )[
                0
            ]  # Shape (m,)

            # no entries met the criteria
            if filtered_in_indices.numel() == 0:
                return None, tensor, mask_tensor

            # Filter where at least one condition is False (instead of both being True)
            # which means these row, col pairs are too close to other motif centers
            filtered_out_indices = torch.nonzero(
                ~torch.all(stacked_valid, dim=1), as_tuple=True
            )[
                0
            ]  # Shape (n-m,)

            filtered_out_rows = valid_rows[
                filtered_out_indices
            ]  # Shape (n-m,) #could be zero
            filtered_out_cols = valid_cols[filtered_out_indices]  # Shape (n-m,)
            # filtered_out are pairs >= threshold. would not be used again

            # Set selected indices to inf
            tensor[filtered_out_rows, filtered_out_cols] = float("inf")
            tensor[filtered_out_cols, filtered_out_rows] = float("inf")

            # update mask
            mask_tensor[filtered_out_rows, filtered_out_cols] = True
            mask_tensor[filtered_out_cols, filtered_out_rows] = True

            # Get corresponding rows and columns
            filtered_in_rows = valid_rows[filtered_in_indices]  # Shape (m,)
            filtered_in_cols = valid_cols[filtered_in_indices]  # Shape (m,)

            # Extract values for (row, col) and (col, row)
            selected_values_first = tensor[
                filtered_in_rows, filtered_in_cols
            ]  # Shape (m,)
            selected_values_second = tensor[
                filtered_in_cols, filtered_in_rows
            ]  # Shape (m,)

            # Stack them together
            stacked_values = torch.stack(
                [selected_values_first, selected_values_second], dim=1
            )  # Shape (m, 2)

            # Find minimum for each pair
            min_values = torch.min(stacked_values, dim=1).values  # Shape (m,)
            # first column is selected_values_first, second is selected_values_first

            # Find the global minimum and its index
            min_value = torch.min(min_values)
            argmin = torch.argmin(min_values)

            chosen_row = filtered_in_rows[argmin]
            chosen_col = filtered_in_cols[argmin]

        new_motif = {"pattern1": profiles[chosen_row], "pattern2": profiles[chosen_col]}

        barycenter = softdtw_barycenter(
            [calculate_g_x(profiles[chosen_row]), calculate_g_x(profiles[chosen_row])],
            gamma=1.0,
            max_iter=5,
            tol=1e-3,
        )  # profiles[row] and profiles[col] are non transformed ts

        motif_info = {
            "motif": new_motif,
            "pair_idx": (chosen_row.cpu().item(), chosen_col.cpu().item()),
            "score": min_value.cpu().item(),
            "barycenter": barycenter,
        }

        return motif_info, tensor, mask_tensor


def all_motifs(profile, masked_arr, gamma, tau):

    motifs = []

    while True:

        # Convert masked array data to a PyTorch tensor
        tensor = torch.tensor(masked_arr.filled(float("inf")), dtype=torch.float32).to(
            "cuda"
        )  # Use 'inf' for masked values

        # Convert the mask to a boolean tensor
        mask_tensor = torch.tensor(masked_arr.mask, dtype=torch.bool).to(
            "cuda"
        )  # mask only, 0 non mask, 1 mask

        motif, tensor, mask_tensor = single_motif(
            motifs,
            profile,
            tensor,
            mask_tensor,
            gamma,
            tau,
        )

        if motif is None:

            break

        row1, row2 = motif["pair_idx"]

        min_score = motif["score"]

        threshold = min_score + tau

        # Get valid indices (entries not masked and not in column `col`)
        valid_cols = torch.arange(tensor.shape[1]).to("cuda")  # Column indices
        valid_mask_cols = (~mask_tensor[row1]) & (
            valid_cols != row2
        )  # Valid entry condition in row views
        valid_rows = torch.arange(tensor.shape[0]).to("cuda")  # Row indices
        valid_mask_rows = (~mask_tensor[:, row1]) & (
            valid_rows != row2
        )  # Valid entry condition in column views

        # Get valid values
        valid_values_cols = tensor[row1, valid_mask_cols]  # selected columns
        valid_values_rows = tensor[valid_mask_rows, row1]  # selected rows

        # Mask entries where value < threshold
        mask_tensor[row1, valid_mask_cols] = valid_values_cols < threshold
        mask_tensor[valid_mask_rows, row1] = valid_values_rows < threshold

        # do the same thing for row2
        valid_cols = torch.arange(tensor.shape[1]).to("cuda")  # Column indices
        valid_mask_cols = (~mask_tensor[row2]) & (
            valid_cols != row1
        )  # Valid entry condition in row views
        valid_rows = torch.arange(tensor.shape[0]).to("cuda")
        valid_mask_rows = (~mask_tensor[:, row2]) & (
            valid_rows != row1
        )  # Valid entry condition in column views
        valid_values_cols = tensor[row2, valid_mask_cols]  # selected columns
        valid_values_rows = tensor[valid_mask_rows, row2]  # selected rows
        mask_tensor[row2, valid_mask_cols] = valid_values_cols < threshold
        mask_tensor[valid_mask_rows, row2] = valid_values_rows < threshold

        # Update masked_arr
        mask_tensor[row1, :] = True
        mask_tensor[:, row1] = True
        mask_tensor[row2, :] = True
        mask_tensor[:, row2] = True

        print(row1, row2)
        motifs.append(motif)

    del tensor
    del mask_tensor

    return motifs

In [ ]:
if __name__ == "__main__":

    train_files = os.listdir("smoothed_multi_train")
    train_list = []
    for train_file in train_files:
        file_path = os.path.join("smoothed_multi_train", train_file)
        train_list.append(np.load(file_path))
    train_data = np.array(train_list)

    # first get masked_arr and save
    # masked_arr = precompute_distances(train_data)
    loaded = np.load("masked_arr_both_multi.npz")
    data = loaded["data"]
    mask = loaded["mask"]
    masked_arr = np.ma.masked_array(data, mask)

    # flatten and fill masked values with np.nan

    flat = np.ma.filled(masked_arr, np.nan).flatten()

    # Get the 20% quantile (i.e., the value below which 20% of the data fall)

    quantile_20 = np.nanquantile(flat, 0.2)

    gamma = quantile_20  # 最大允许距离阈值
    tau = [gamma * 0.5]
    # tau = [gamma * 0.3, gamma * 0.4, gamma * 0.5]  # 容忍值

    motifs_list = []

    for t in tau:

        # 并行计算
        motifs = all_motifs(train_data, masked_arr, gamma, t)
        motifs_list.append(motifs)

        # Save motifs to a file
        with open(f"{gamma}_{t}_motifs.pkl", "wb") as f:
            pickle.dump(motifs, f)

2605 2701
3446 3452
156 1156
69 1116
1377 3727
330 1227
1908 2350
49 1429
1431 1935
2052 3739
837 849
258 3737
2300 2444
670 3770
1947 1956
265 1984
2286 3750
845 4124
